# DataFrames as RDD
in chapter 1 we learned about Pandas DataFrames and how great they are to work with. Let's see how we can make similar objects with Spark.

## Spark SQL
In the previous sheet, we used a basic SparkContext to access our data. But what if we wanted a more robust Spark interface that could interact with SQL, Hive, and other data storage systems? That's where SparkSession comes into the picture. Part of the `pyspark.sql` package, this constructor can be connected to a wide variaty of data sources and returns a SparkSession which can be used to create RDDs. Luckily, one of the data sources is your local machine so we can access the flight csv similarly.

In [1]:
import pyspark
sparkql = pyspark.sql.SparkSession.builder.master('local').getOrCreate()

- `SparkSession.builder` is creating our session
- `.master('local')` tells the Session
- `getOrCreate()` creates a new Session since none has been created already. The builder can retrieve existing session with this function as well.

### Read flights
Now that we've got a session, let's read in the same flight data as before!

In [2]:
flight_file = '../data/flights.csv'
flight_df = sparkql.read.csv(flight_file, header=True)
print(flight_df.columns)
print(flight_df.schema)
flight_df.show(3)

['flight_date', 'airline', 'tailnumber', 'flight_number', 'src', 'dest', 'departure_time', 'arrival_time', 'flight_time', 'distance']
StructType(List(StructField(flight_date,StringType,true),StructField(airline,StringType,true),StructField(tailnumber,StringType,true),StructField(flight_number,StringType,true),StructField(src,StringType,true),StructField(dest,StringType,true),StructField(departure_time,StringType,true),StructField(arrival_time,StringType,true),StructField(flight_time,StringType,true),StructField(distance,StringType,true)))
+-----------+-------+----------+-------------+---+----+--------------+------------+-----------+--------+
|flight_date|airline|tailnumber|flight_number|src|dest|departure_time|arrival_time|flight_time|distance|
+-----------+-------+----------+-------------+---+----+--------------+------------+-----------+--------+
| 2019-11-28|     9E|    N8974C|         3280|CHA| DTW|          1300|        1455|      115.0|   505.0|
| 2019-11-28|     9E|    N901XJ|   

In [3]:
print(flight_df.count())

12715


## DataFrame operation with an RDD
The syntax is very similar but now we are using the `.read.csv` method which functions very similar to the `read_csv()` method from `pandas` creating an RDD with DataFrame-like properties as we can see from the `schema`, `columns` and `head` above. As such, let try some of the DataFrame operations!
### Change column data type
As we can see from the `flight_df.schema` since no schema information was supplied, everything was read in as `StringType`. Let's change the date columns to the correct type.

In [4]:
from pyspark.sql.types import DateType
flight_df = flight_df.withColumn('flight_date', flight_df['flight_date'].cast(DateType()))
print(flight_df.schema)

StructType(List(StructField(flight_date,DateType,true),StructField(airline,StringType,true),StructField(tailnumber,StringType,true),StructField(flight_number,StringType,true),StructField(src,StringType,true),StructField(dest,StringType,true),StructField(departure_time,StringType,true),StructField(arrival_time,StringType,true),StructField(flight_time,StringType,true),StructField(distance,StringType,true)))


These also allows us to see the power of the `withColumn` function. the first argument is the 
output column name. If this is an existing column in the RDD this allows us to update columns. If it is a new column name, a new column is created:

In [5]:
from pyspark.sql.types import DoubleType
flight_df = flight_df.withColumn('num_dist', flight_df['distance'].cast(DoubleType()))
flight_df.show(3)

+-----------+-------+----------+-------------+---+----+--------------+------------+-----------+--------+--------+
|flight_date|airline|tailnumber|flight_number|src|dest|departure_time|arrival_time|flight_time|distance|num_dist|
+-----------+-------+----------+-------------+---+----+--------------+------------+-----------+--------+--------+
| 2019-11-28|     9E|    N8974C|         3280|CHA| DTW|          1300|        1455|      115.0|   505.0|   505.0|
| 2019-11-28|     9E|    N901XJ|         3281|JAX| RDU|           700|         824|       84.0|   407.0|   407.0|
| 2019-11-28|     9E|    N901XJ|         3282|RDU| LGA|           900|        1039|       99.0|   431.0|   431.0|
+-----------+-------+----------+-------------+---+----+--------------+------------+-----------+--------+--------+
only showing top 3 rows



Similarly, it is easy to drop columns as well:

In [6]:
flight_df = flight_df.drop("distance")
flight_df.show(3)

+-----------+-------+----------+-------------+---+----+--------------+------------+-----------+--------+
|flight_date|airline|tailnumber|flight_number|src|dest|departure_time|arrival_time|flight_time|num_dist|
+-----------+-------+----------+-------------+---+----+--------------+------------+-----------+--------+
| 2019-11-28|     9E|    N8974C|         3280|CHA| DTW|          1300|        1455|      115.0|   505.0|
| 2019-11-28|     9E|    N901XJ|         3281|JAX| RDU|           700|         824|       84.0|   407.0|
| 2019-11-28|     9E|    N901XJ|         3282|RDU| LGA|           900|        1039|       99.0|   431.0|
+-----------+-------+----------+-------------+---+----+--------------+------------+-----------+--------+
only showing top 3 rows



If you only want to rename a column with no changes, we can use `withColumnRenamed` instead of a full `withColumn`:

In [7]:
flight_df = flight_df.withColumnRenamed("num_dist", "dist")
flight_df.show(3)

+-----------+-------+----------+-------------+---+----+--------------+------------+-----------+-----+
|flight_date|airline|tailnumber|flight_number|src|dest|departure_time|arrival_time|flight_time| dist|
+-----------+-------+----------+-------------+---+----+--------------+------------+-----------+-----+
| 2019-11-28|     9E|    N8974C|         3280|CHA| DTW|          1300|        1455|      115.0|505.0|
| 2019-11-28|     9E|    N901XJ|         3281|JAX| RDU|           700|         824|       84.0|407.0|
| 2019-11-28|     9E|    N901XJ|         3282|RDU| LGA|           900|        1039|       99.0|431.0|
+-----------+-------+----------+-------------+---+----+--------------+------------+-----------+-----+
only showing top 3 rows



### Selection
We can also select specific columns to return or fitler rows similar to how we filtered the simple RDDs


In [8]:
flight_df.select("flight_date", "airline", "dest").show(3)
flight_df.filter(flight_df.src == "PDX").show(3)
flight_df.filter((flight_df.src == "PDX") | (flight_df.dest == "PDX")).show(3)

+-----------+-------+----+
|flight_date|airline|dest|
+-----------+-------+----+
| 2019-11-28|     9E| DTW|
| 2019-11-28|     9E| RDU|
| 2019-11-28|     9E| LGA|
+-----------+-------+----+
only showing top 3 rows

+-----------+-------+----------+-------------+---+----+--------------+------------+-----------+------+
|flight_date|airline|tailnumber|flight_number|src|dest|departure_time|arrival_time|flight_time|  dist|
+-----------+-------+----------+-------------+---+----+--------------+------------+-----------+------+
| 2019-11-28|     AA|    N832NN|         1402|PDX| PHX|           820|        1153|      153.0|1009.0|
| 2019-11-28|     AA|    N939AN|         2298|PDX| ORD|           627|        1229|      242.0|1739.0|
| 2019-11-28|     AA|    N992AU|         2577|PDX| DFW|           600|        1141|      221.0|1616.0|
+-----------+-------+----------+-------------+---+----+--------------+------------+-----------+------+
only showing top 3 rows

+-----------+-------+----------+--------

### Aggregation
Count arrivals per airport. Then arrivals per airline per airport.

In [9]:
flight_df.groupBy("dest").count().show(3)
flight_df.groupBy("dest", "airline").count().show(3)

+----+-----+
|dest|count|
+----+-----+
| BGM|    1|
| PSE|    3|
| INL|    1|
+----+-----+
only showing top 3 rows

+----+-------+-----+
|dest|airline|count|
+----+-------+-----+
| MDT|     9E|    1|
| KTN|     AS|    4|
| JAX|     EV|    2|
+----+-------+-----+
only showing top 3 rows

